In [ ]:
import pandas as pd

# LOAD DELPRO CALVING DATA
df = pd.read_csv("Del_Calving240531.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df.shape)  # 9917, 10col
df = df.sort_values(by=["SE_Number", "CalvingDate"])
df2 = df.drop_duplicates(subset=["SE_Number", "CalvingDate"])
print(df2.shape)  # 9710, 10col

# Find earliest and latest data points for comparison
earliest_date = df2['CalvingDate'].min()
print(f"The earliest calving date in DelPro is: {earliest_date}")  # 2018-04-07

last_date = df2['CalvingDate'].max()
print(f"The last calving date in DelPro is: {last_date}")  # 2024-05-30

# Rename and keep only relevant columns
df2 = df2.copy()  # Ensuring we're working with a copy
df2.loc[:, "CalvingDate_DelPro"] = df2["CalvingDate"]
df2 = df2.rename(columns={"CalvingEase": "CalvingEase_DelPro", "CalvingSireBullID": "CalvingSireBullID_DelPro"})

col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "CalvingDate_DelPro", "CalvingEase_DelPro",
            "CalvingSireBullID_DelPro"]
df2 = df2[col_keep]
df2.to_csv("comparison.csv", index=False)

# LOAD COW DATABASE CALVING DATA
df3 = pd.read_csv("Kok_Calving240807.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df3.shape)  # 25,761, 16col
df3 = df3.sort_values(by=["BirthID", "CalvingDate"])
df4 = df3.drop_duplicates(subset=["BirthID", "CalvingDate"])
print(df4.shape)  # 23,082, 16col

# Find earliest and latest data points for comparison
earliest_date = df4['CalvingDate'].min()
print(f"The earliest calving date in cow database is: {earliest_date}")  # 2008-04-28

last_date = df4['CalvingDate'].max()
print(f"The last calving date in cow database is: {last_date}")  # 2024-02-27

df4 = df4.copy()  # Ensuring we're working with a copy
df4.loc[:, "CalvingDate_Kok"] = df4["CalvingDate"]
df4 = df4.rename(columns={"ActiveHerdNumber": "FarmName_Pseudo", "BirthID": "SE_Number",
                          "CalvingNumber": "CalvingNumber_Kok", "CalvingEase": "CalvingEase_Kok"})

col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "CalvingDate_Kok", "CalvingNumber_Kok", "CalvingEase_Kok"]
df4 = df4[col_keep]

df5 = df2.merge(df4, on=["FarmName_Pseudo", "SE_Number", "CalvingDate"], how="outer")
df5 = df5.sort_values(by=["SE_Number", "CalvingDate"])
count_date = df5["CalvingDate"].count()
print(f"Number of non-missing calving events: {count_date} observations")  # 26,228 obs
total_date = df5["CalvingDate"].size
print(f"Total number of calving events: {total_date} observations")  # 26,228 obs
df5.to_csv("comparison_calving.csv", index=False)

# FIND START AND STOP OF HERDS RECORDING TO GIGACOW
dfx = pd.read_csv("Del_Calving240531.csv", delimiter=';', low_memory=False)
dfx = dfx.sort_values(by=["FarmName_Pseudo", "CalvingDate"])

first_dates = dfx.groupby(["FarmName_Pseudo"])["CalvingDate"].min().reset_index()
first_dates = first_dates.rename(columns={"CalvingDate": "CalvingDate_first"})
last_dates = dfx.groupby(["FarmName_Pseudo"])["CalvingDate"].max().reset_index()
last_dates = last_dates.rename(columns={"CalvingDate": "CalvingDate_last"})

dfx3 = first_dates.merge(last_dates, on=["FarmName_Pseudo"], how="left")
dfx2 = pd.read_csv("comparison_calving.csv", low_memory=False)
dfx4 = dfx3.merge(dfx2, on=["FarmName_Pseudo"], how="left")
dfx4 = dfx4.sort_values(by=["SE_Number", "CalvingDate"])


def data(row):
    if row["CalvingDate_first"] <= row["CalvingDate"] <= row["CalvingDate_last"]:
        return 1
    else:
        return 0


dfx4["data_incl"] = dfx4.apply(data, axis=1)
dfx4 = dfx4[dfx4["data_incl"] == 1]
print(f"Structure of calving events data: {dfx4.shape} observations")  # 13,108 obs, 12col

# Compare "CalvingDate_DelPro" with "CalvingDate_Kok"
count1 = dfx4["CalvingDate_DelPro"].count()
print(f"Number of calving events in DelPro: {count1} observations")  # 9,710 obs
count2 = dfx4["CalvingDate_Kok"].count()
print(f"Number of calving events in cow database: {count2} observations")  # 9,962 obs
count3 = dfx4[["CalvingDate_DelPro", "CalvingDate_Kok"]].dropna().shape[0]
print(f"Number of calving events in both DelPro and cow database i.e. similar date: {count3} observations")  # 6,564 obs

dfx4["CalvingDate_DelPro"] = pd.to_datetime(dfx4["CalvingDate_DelPro"])
dfx4["CalvingDate_Kok"] = pd.to_datetime(dfx4["CalvingDate_Kok"])
dfx4['Days_diff'] = (dfx4['CalvingDate_DelPro'] - dfx4['CalvingDate_Kok']).dt.days
dfx4a = dfx4[dfx4['Days_diff'] != 0]
print(f"Days difference in DelPro and cow database: {dfx4a.shape}")  # 6,544, 13obs

condition = dfx4["CalvingDate_DelPro"].notna() & dfx4["CalvingDate_Kok"].isna()
count4 = dfx4[condition].shape[0]
print(f"Number of calving events in DelPro but not in cow database: {count4} observations")  # 3,146 obs

condition = dfx4["CalvingDate_Kok"].notna() & dfx4["CalvingDate_DelPro"].isna()
count5 = dfx4[condition].shape[0]
print(f"Number of calving events not in DelPro but in cow database: {count5} observations")  # 3,398 obs

dfx4.to_csv("comparison_calving2.csv", index=False)

In [ ]:
# LOAD DELPRO CULLING DATA
df6 = pd.read_csv("Del_Cow240531.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df6.shape)  # 24,473, 42col
df6 = df6.sort_values(by=["SE_Number", "CullDecisionDate"])
df7 = df6.drop_duplicates(subset=["SE_Number", "CullDecisionDate"])
print(df7.shape)  # 24,199, 42col

# Count notna for CullDecisionDate, DeathDate and ExitDate
non_missing_count = df7["CullDecisionDate"].count()
print(f"Number of non-missing values in CullDecisionDate: {non_missing_count}")  # 7070

non_missing_count = df7["DeathDate"].count()
print(f"Number of non-missing values in DeathDate: {non_missing_count}")  # 0

non_missing_count = df7["ExitDate"].count()
print(f"Number of non-missing values in ExitDate: {non_missing_count}")  # 0

# Find earliest and latest data points for comparison
print(df7.dtypes)
df7 = df7.copy()
df7["CullDecisionDate"] = pd.to_datetime(df7["CullDecisionDate"])
earliest_date = df7["CullDecisionDate"].min()
print(f"The earliest culling date in DelPro is: {earliest_date}")  # 2010-09-02

last_date = df7["CullDecisionDate"].max()
print(f"The last culling date in DelPro is: {last_date}")  # 2024-06-02

col_keep = ["FarmName_Pseudo", "SE_Number", "CullDecisionDate", "CullReason1", "CullReason2"]
df7 = df7[col_keep]
df7.loc[:, "ExitDate_DelPro"] = df7["CullDecisionDate"]
df7 = df7.rename(columns={"CullDecisionDate": "ExitDate",
                          "CullReason1": "ExitReason_PrimaryReason_DelPro",
                          "CullReason2": "ExitReason_SecondaryReason1_DelPro"})
df7.to_csv("comparison.csv", index=False)

# LOAD COW DATABASE CULLING/EXIT DATA
df8 = pd.read_csv("Kok_HerdEntryExit240807.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df8.shape)  # 32,626, 20col
df8 = df8.sort_values(by=["BirthID", "ExitDate"])
df9 = df8.drop_duplicates(subset=["BirthID", "ExitDate"])
print(df9.shape)  # 32,615, 20col

# Find earliest and latest data points for comparison
print(df9.dtypes)
df9 = df9.copy()
df9["ExitDate"] = pd.to_datetime(df9["ExitDate"])
earliest_date = df9["ExitDate"].min()
print(f"The earliest culling date in cow database is: {earliest_date}")  # 2007-10-12

last_date = df9["ExitDate"].max()
print(f"The last culling date in cow database is: {last_date}")  # 2024-02-29

col_keep = ["ActiveHerdNumber", "BirthID", "EntryDate", "ExitDate", "ExitReason_PrimaryReason",
            "ExitReason_SecondaryReason1", "ExitReason_SecondaryReason2"]
df9 = df9[col_keep]
df9.loc[:, "ExitDate_Kok"] = df9["ExitDate"]
df9 = df9.rename(columns={"BirthID": "SE_Number",
                          "ActiveHerdNumber": "FarmName_Pseudo",
                          "EntryDate": "EntryDate_Kok",
                          "ExitReason_PrimaryReason": "ExitReason_PrimaryReason_Kok",
                          "ExitReason_SecondaryReason1": "ExitReason_SecondaryReason1_Kok",
                          "ExitReason_SecondaryReason2": "ExitReason_SecondaryReason2_Kok"})
df9.to_csv("comparison2.csv", index=False)

df10 = df7.merge(df9, on=["FarmName_Pseudo", "SE_Number", "ExitDate"], how="outer")
df10 = df10.sort_values(by=["SE_Number", "ExitDate"])

count_date = df10["ExitDate"].count()
print(f"Number of non-missing culling/exit records: {count_date} observations")  # 30,586 obs
total_count = df10["ExitDate"].size
print(f"Total number of culling/exit records: {total_count} observations, i.e. 20k living animals")  # 49,476 obs
df10.to_csv("comparison_entry_exit.csv", index=False)

# FIND START AND STOP OF HERDS RECORDING TO GIGACOW
dfx5 = pd.read_csv("Del_Cow240531.csv", delimiter=';', low_memory=False)
dfx6 = dfx5.sort_values(by=["FarmName_Pseudo", "CullDecisionDate"])
dfx6["ExitDate"] = pd.to_datetime(dfx6["ExitDate"])

first_dates = dfx6.groupby(["FarmName_Pseudo"])["ExitDate"].min().reset_index()
first_dates = first_dates.rename(columns={"ExitDate": "ExitDate_first"})
last_dates = dfx6.groupby(["FarmName_Pseudo"])["ExitDate"].max().reset_index()
last_dates = last_dates.rename(columns={"ExitDate": "ExitDate_last"})

dfx7 = first_dates.merge(last_dates, on=["FarmName_Pseudo"], how="left")
dfx8 = pd.read_csv("comparison_entry_exit.csv", low_memory=False)
dfx9 = dfx7.merge(dfx8, on=["FarmName_Pseudo"], how="left")
dfx9 = dfx9.sort_values(by=["SE_Number", "ExitDate"])
dfx9["ExitDate_first"] = pd.to_datetime(dfx9["ExitDate_first"])
dfx9["ExitDate_last"] = pd.to_datetime(dfx9["ExitDate_last"])


def data(row):
    if pd.notna(row["ExitDate"]):
        # Ensure ExitDate_first and ExitDate_last are valid dates before comparing
        if pd.notna(row["ExitDate_first"]) and pd.notna(row["ExitDate_last"]):
            if row["ExitDate_first"] <= row["ExitDate"] <= row["ExitDate_last"]:
                return 1
            else:
                return 0
        else:
            return 1  # cuz still alive
    else:
        return 1  # cuz still alive


dfx9["data_incl"] = dfx9.apply(data, axis=1)
dfx9 = dfx9[dfx9["data_incl"] == 1]
print(f"Structure of culling data: {dfx9.shape} observations")  # 40,352 obs, 14col

# Count occurrences
count1 = dfx9["ExitDate_DelPro"].count()
print(f"Number of culling/exit records in DelPro: {count1} observations")  # 7070 obs
count2 = dfx9["ExitDate_Kok"].count()
print(f"Number of culling/exit records in cow database: {count2} observations")  # 18,864 obs
count3 = dfx9[["ExitDate_DelPro", "ExitDate_Kok"]].dropna().shape[0]
print(f"Number of culling/exit records in both DelPro and cow database: {count3} observations")  # 3,188 obs

condition = dfx9["ExitDate_DelPro"].notna() & dfx9["ExitDate_Kok"].isna()
count4 = dfx9[condition].shape[0]
print(f"Number of culling/exit records in DelPro but not in cow database: {count4} observations")  # 3,882 obs

condition = dfx9["ExitDate_Kok"].notna() & dfx9["ExitDate_DelPro"].isna()
count5 = dfx9[condition].shape[0]
print(f"Number of culling/exit records not in DelPro but in cow database: {count5} observations")  # 15,676 obs

dfx9.to_csv("comparison_entry_exit2.csv", index=False)

In [ ]:
# LOAD DELPRO INSEMINATION DATA
df11 = pd.read_csv("Del_Insemination240531.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df11.shape)  # 18,775, 9col
df11 = df11.sort_values(by=["SE_Number", "InseminationDate"])
df12 = df11.drop_duplicates(subset=["SE_Number", "InseminationDate"])
print(df12.shape)  # 18,689, 9col

# Find earliest and latest data points for comparison
earliest_date = df12["InseminationDate"].min()
print(f"The earliest insemination date in DelPro is: {earliest_date}")  # 2018-06-08

last_date = df12["InseminationDate"].max()
print(f"The last insemination date in DelPro is: {last_date}")  # 2024-05-30

col_keep = ["FarmName_Pseudo", "SE_Number", "InseminationDate", "Breeder"]
df12 = df12[col_keep]
df12.loc[:, "InseminationDate_DelPro"] = df12["InseminationDate"]
df12 = df12.rename(columns={"Breeder": "Breeder_DelPro"})
df12.to_csv("comparison.csv", index=False)

# LOAD COW DATABASE INSEMINATION DATA
df13 = pd.read_csv("Kok_Reproduction240807.csv", delimiter=';', low_memory=False)

# Create a boolean mask where SireBull_SE_Number is NE 0
mask = df13["SireBull_SE_Number"] != 0
# Sum the mask to count the number of True values (i.e., the number of not 0s)
count_non_zeros = mask.sum()
print(f"Number of events with sire ID in cow database: {count_non_zeros}")  # 91,840

# Count occurrences of each unique value in the EventType column
value_counts = df13["EventType"].value_counts()
print(value_counts)
"""
EventType
Inseminering               46333
Dräktighetsundersökning    39941
Behandling                  2008
Embryoinlägg                1960
Betäckning                  1429
Fri bet                      169
"""

# Keep only data from inseminations
df14 = df13[df13["EventType"] == "Inseminering"]

# Check for duplicates and sort
print(df14.shape)  # 46,333, 27col
df14 = df14.sort_values(by=["BirthID", "EventDate"])
df15 = df14.drop_duplicates(subset=["BirthID", "EventDate"])
print(df15.shape)  # 42,583, 27col

# Find earliest and latest data points for comparison
earliest_date = df15["EventDate"].min()
print(f"The earliest insemination date in cow database is: {earliest_date}")  # 2009-05-13

last_date = df15["EventDate"].max()
print(f"The last insemination date in cow database is: {last_date}")  # 2024-02-29

col_keep = ["ActiveHerdNumber", "BirthID", "EventDate", "Estrus", "SireBull_SE_Number"]
df15 = df15[col_keep]
df15.loc[:, "InseminationDate_Kok"] = df15["EventDate"]
df15 = df15.rename(columns={"BirthID": "SE_Number",
                            "ActiveHerdNumber": "FarmName_Pseudo",
                            "Estrus": "Estrus_Kok",
                            "EventDate": "InseminationDate",
                            "SireBull_SE_Number": "SireBull_SE_Number_Kok"})
df15.to_csv("comparison2.csv", index=False)

df16 = df12.merge(df15, on=["FarmName_Pseudo", "SE_Number", "InseminationDate"], how="outer")
df16 = df16.sort_values(by=["SE_Number", "InseminationDate"])

count_date = df16["InseminationDate"].count()
print(f"Number of non-missing insemination events: {count_date} observations")  # 48,871 obs
total_date = df16["InseminationDate"].size
print(f"Total number of insemination events: {total_date} observations")  # 48,871 obs
df16.to_csv("comparison_insemination.csv", index=False)

# FIND START AND STOP OF HERDS RECORDING TO GIGACOW
dfx10 = pd.read_csv("Del_Insemination240531.csv", delimiter=';', low_memory=False)
dfx10 = dfx10.sort_values(by=["FarmName_Pseudo", "InseminationDate"])

first_dates = dfx10.groupby(["FarmName_Pseudo"])["InseminationDate"].min().reset_index()
first_dates = first_dates.rename(columns={"InseminationDate": "InseminationDate_first"})
last_dates = dfx10.groupby(["FarmName_Pseudo"])["InseminationDate"].max().reset_index()
last_dates = last_dates.rename(columns={"InseminationDate": "InseminationDate_last"})

dfx11 = first_dates.merge(last_dates, on=["FarmName_Pseudo"], how="left")
dfx12 = pd.read_csv("comparison_insemination.csv", low_memory=False)
dfx13 = dfx11.merge(dfx12, on=["FarmName_Pseudo"], how="left")
dfx13 = dfx13.sort_values(by=["SE_Number", "InseminationDate"])
dfx13["InseminationDate"] = pd.to_datetime(dfx13["InseminationDate"])
dfx13["InseminationDate_first"] = pd.to_datetime(dfx13["InseminationDate_first"])
dfx13["InseminationDate_last"] = pd.to_datetime(dfx13["InseminationDate_last"])


def data2(row):
    if row["InseminationDate_first"] <= row["InseminationDate"] <= row["InseminationDate_last"]:
        return 1
    else:
        return 0


dfx13["data_incl"] = dfx13.apply(data2, axis=1)
dfx13 = dfx13[dfx13["data_incl"] == 1]
print(f"Structure of insemination data: {dfx13.shape} observations")  # 40,352 obs, 14col

# Count occurrences
count_date1 = dfx13["InseminationDate_DelPro"].count()
print(f"Number of insemination events in DelPro: {count_date1} observations")  # 7070 obs
count_date2 = dfx13["InseminationDate_Kok"].count()
print(f"Number of insemination events in cow database: {count_date2} observations")  # 18,864 obs
count_date3 = dfx13[["InseminationDate_DelPro", "InseminationDate_Kok"]].dropna().shape[0]
print(f"Number of insemination events in both DelPro and cow database: {count_date3} observations")  # 3,188 obs

condition = dfx13["InseminationDate_DelPro"].notna() & dfx13["InseminationDate_Kok"].isna()
count4 = dfx13[condition].shape[0]
print(f"Number of insemination records in DelPro but not in cow database: {count4} observations")  # 3,882 obs

condition = dfx13["InseminationDate_Kok"].notna() & dfx13["InseminationDate_DelPro"].isna()
count5 = dfx13[condition].shape[0]
print(f"Number of insemination records not in DelPro but in cow database: {count5} observations")  # 15,676 obs

dfx13.to_csv("comparison_insemination2.csv", index=False)

In [ ]:
# LOAD PREGNANCY CHECK DATA FROM DELPRO
df17 = pd.read_csv("Del_PregnancyCheck240531.csv", delimiter=';', low_memory=False)

# Check for duplicates and sort
print(df17.shape)  # 13,315, 9col
df17 = df17.sort_values(by=["SE_Number", "PregnancyCheckDate"])
df18 = df17.drop_duplicates(subset=["SE_Number", "PregnancyCheckDate"])
print(df18.shape)  # 13,292, 9col

# Find earliest and latest data points for comparison
earliest_date = df18["PregnancyCheckDate"].min()
print(f"The earliest pregnancy check date in DelPro is: {earliest_date}")  # 2018-10-16

last_date = df18["PregnancyCheckDate"].max()
print(f"The last pregnancy check date in DelPro is: {last_date}")  # 2024-05-28

col_keep = ["FarmName_Pseudo", "SE_Number", "PregnancyCheckDate", "PregnancyCheckResult"]
df18 = df18[col_keep]
df18.loc[:, "PregnancyCheckDate_DelPro"] = df18["PregnancyCheckDate"]
df18 = df18.rename(columns={"PregnancyCheckResult": "PregnancyCheckResult_DelPro"})
df18.to_csv("comparison.csv", index=False)

# LOAD PREGNANCY CHECK DATA FROM COW DATABASE
df19 = pd.read_csv("Kok_Reproduction240807.csv", delimiter=';', low_memory=False)

# Keep only data from inseminations
df19 = df19[df19["EventType"] == "Dräktighetsundersökning"]

# Check for duplicates and sort
print(df19.shape)  # 39,941, 27col
df19 = df19.sort_values(by=["BirthID", "EventDate"])
df20 = df19.drop_duplicates(subset=["BirthID", "EventDate"])
print(df20.shape)  # 36,908, 27col

# Find earliest and latest data points for comparison
earliest_date = df20["EventDate"].min()
print(f"The earliest pregnancy check date in cow database is: {earliest_date}")  # 2009-05-13

last_date = df20["EventDate"].max()
print(f"The last pregnancy check date in cow database is: {last_date}")  # 2024-02-29

# Count occurrences of each unique value in PregnancyStatus column
value_counts = df20["PregnancyStatus"].value_counts()
print(value_counts)
"""
PregnancyStatus
Dräktig Analys Dr A                16372
Dräktig (undersökt) Dr             11250
Ej Dräktig (undersökt) eDr          3240
Ej dräktig Analys eDr A             2964
Dräktig ? Analys Dr? A               938
Dräktig ? (undersökt) Dr?            330
Dräktig (tjurbetäckt) Dr             244
Dräktig (ej undersökt) Dr             96
Ej dräktig (tjurbetäckt) eDr          51
Ej Dräktig (ej undersökt) eDr         46
Dräktig ? (tjurbetäckt) Dr?           28
Dräktig, (sem/bet annan bes) Dr        2
"""

col_keep = ["ActiveHerdNumber", "BirthID", "EventDate", "PregnancyStatus"]
df20 = df20[col_keep]
df20.loc[:, "PregnancyCheckDate_Kok"] = df20["EventDate"]
df20 = df20.rename(columns={"BirthID": "SE_Number",
                            "ActiveHerdNumber": "FarmName_Pseudo",
                            "EventDate": "PregnancyCheckDate",
                            "PregnancyStatus": "PregnancyStatus_Kok"})
df20.to_csv("comparison2.csv", index=False)

df21 = df18.merge(df20, on=["FarmName_Pseudo", "SE_Number", "PregnancyCheckDate"], how="outer")
df21 = df21.sort_values(by=["SE_Number", "PregnancyCheckDate"])

count_date = df21["PregnancyCheckDate"].count()
print(f"Number of non-missing pregnancy check events: {count_date} observations")  # 45,124 obs
total_date = df21["PregnancyCheckDate"].size
print(f"Total number of pregnancy check events: {total_date} observations")  # 45,124 obs
df21.to_csv("comparison_preg_check.csv", index=False)

# FIND START AND STOP OF HERDS RECORDING TO GIGACOW
dfx14 = pd.read_csv("Del_PregnancyCheck240531.csv", delimiter=';', low_memory=False)
dfx14 = dfx14.sort_values(by=["FarmName_Pseudo", "PregnancyCheckDate"])

first_dates = dfx14.groupby(["FarmName_Pseudo"])["PregnancyCheckDate"].min().reset_index()
first_dates = first_dates.rename(columns={"PregnancyCheckDate": "PregnancyCheckDate_first"})
last_dates = dfx14.groupby(["FarmName_Pseudo"])["PregnancyCheckDate"].max().reset_index()
last_dates = last_dates.rename(columns={"PregnancyCheckDate": "PregnancyCheckDate_last"})

dfx15 = first_dates.merge(last_dates, on=["FarmName_Pseudo"], how="left")
dfx16 = pd.read_csv("comparison_preg_check.csv", low_memory=False)
dfx17 = dfx15.merge(dfx16, on=["FarmName_Pseudo"], how="left")
dfx17 = dfx17.sort_values(by=["SE_Number", "PregnancyCheckDate"])
dfx17["PregnancyCheckDate"] = pd.to_datetime(dfx17["PregnancyCheckDate"])
dfx17["PregnancyCheckDate_first"] = pd.to_datetime(dfx17["PregnancyCheckDate_first"])
dfx17["PregnancyCheckDate_last"] = pd.to_datetime(dfx17["PregnancyCheckDate_last"])


def data2(row):
    if row["PregnancyCheckDate_first"] <= row["PregnancyCheckDate"] <= row["PregnancyCheckDate_last"]:
        return 1
    else:
        return 0


dfx17["data_incl"] = dfx17.apply(data2, axis=1)
dfx17 = dfx17[dfx17["data_incl"] == 1]
print(f"Structure of pregnancy check data: {dfx17.shape} observations")  # 25,569 obs, 10col

# Count occurrences
count1 = dfx17["PregnancyCheckDate_DelPro"].count()
print(f"Number of pregnancy check in DelPro: {count1} observations")  # 13,292 obs
count2 = dfx17["PregnancyCheckDate_Kok"].count()
print(f"Number of pregnancy check in cow database: {count2} observations")  # 17,353 obs
count3 = dfx17[["PregnancyCheckDate_DelPro", "PregnancyCheckDate_Kok"]].dropna().shape[0]
print(f"Number of pregnancy checks in both DelPro and cow database: {count3} observations")  # 5,076 obs

condition = dfx17["PregnancyCheckDate_DelPro"].notna() & dfx17["PregnancyCheckDate_Kok"].isna()
count4 = dfx17[condition].shape[0]
print(f"Number of pregnancy check records in DelPro but not in cow database: {count4} observations")  # 8,216 obs

condition = dfx17["PregnancyCheckDate_Kok"].notna() & dfx17["PregnancyCheckDate_DelPro"].isna()
count5 = dfx17[condition].shape[0]
print(f"Number of pregnancy check records not in DelPro but in cow database: {count5} observations")  # 12,277 obs

dfx17.to_csv("comparison_preg_check2.csv", index=False)

In [ ]:
# ADD ARRIVAL DATE FROM DEL_COW TO CULLING FILE TO CORRESPOND WITH ENTRY DATE FROM COW DATABASE?

In [ ]:
# LOAD COW DATABASE BREED INFO
df = pd.read_csv("Kok_Lineage240821.csv", delimiter=';', low_memory=False)
df.rename(columns={"BirthID": "SE_Number", "ActiveHerdNumber": "FarmName_Pseudo", "Father_Breed": "SireBreedKok",
                   "Mother_Breed": "DamBreedKok", "MothersFather_Breed": "MGSBreedKok"}, inplace=True)
col_keep = ["FarmName_Pseudo", "SE_Number", "SireBreedKok", "DamBreedKok", "MGSBreedKok"]
df = df[col_keep]

# Check for duplicates and sort
print(df.shape)  # 24,067
df = df.sort_values(by=["FarmName_Pseudo", "SE_Number"])
df2 = df.drop_duplicates(subset=["SE_Number", "SireBreedKok", "DamBreedKok", "MGSBreedKok"])
print(f"No. cows with breed data: {df2.shape}")  # 22,653 => 1414 duplicates, all okay
df2 = df.drop_duplicates(subset=["SE_Number"])
print(f"No. cows with different breed data recorded: {df2.shape}")  # 22,607 => 46 cows with conflicting breed
df2.to_csv("kok_ras.csv", index=False)

""" The 46 cows with conflicting breed are a mess, e.g. LIM and SJB for the same sire, remove!
duplicate_records = df2[df2['SE_Number'].duplicated(keep=False)]
duplicate_records = duplicate_records.copy()
duplicate_records["DupBreed"] = 1
col_keep = ["SE_Number", "DupBreed"]
duplicate_records = duplicate_records[col_keep]
duplicate_records = duplicate_records.drop_duplicates(subset=["SE_Number"])
duplicate_records.to_csv("kok_ras.csv", index=False)

df = df2.merge(duplicate_records, on=["SE_Number"], how="left")
df.to_csv("kok_ras.csv", index=False)
"""

# Add breed info from DelPro
df3 = pd.read_csv("Del_Cow240531.csv", delimiter=';', low_memory=False)
df3.rename(columns={"BreedName": "BreedDelPro"}, inplace=True)
col_keep = ["SE_Number", "BreedDelPro"]  # "BreedPart1", "BreedPart2", "BreedPart3", "BreedPart4"
df3 = df3[col_keep]
df4 = df3.drop_duplicates(subset=["SE_Number", "BreedDelPro"])
print(f"No. cows with breed data in DelPro: {df4.shape}")  # 23,794
df4 = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows with unique breed data in DelPro: {df4.shape}")  # 23,572 => 222 conflicting records

"""
# Are BreedPartN used in DelPro? - No, all empty
frequency_table = df4.groupby(['BreedPart4']).size().reset_index(name='Frequency')
print(frequency_table)
"""

# Merge and keep only where have data from both sources
df5 = df2.merge(df4, on=["SE_Number"], how="left")
df5 = df5[pd.notna(df5["BreedDelPro"])]
print(f"No. cows with breed data in both DelPro and Cow database: {df5.shape}")  # 11,085
df5.to_csv("comparison_breed.csv", index=False)

# Look at different breed inclusion
filtered_df = df5[df5['BreedDelPro'] == "01 SRB"]
frequency_table = filtered_df.groupby(['SireBreedKok']).size().reset_index(name='Frequency')
print(frequency_table)
"""
SireBreedKok  Frequency
0           ANG         92
1           BGA          1
2            BL          5
3           CHA         27
4           HER         29
5           LIM         81
6           MON          9
7            RB        465
8        RB-SRB         18
9           SAB          3
10          SIM         10
11          SJB          2
12          SLB        378
13          SRB       1152
14       SRB-RB          1
15          WAG          5
"""
frequency_table = filtered_df.groupby(['DamBreedKok']).size().reset_index(name='Frequency')
print(frequency_table)
"""
DamBreedKok  Frequency
0      MON-SLB         13
1      MON-SRB          2
2           RB         62
3       RB-HER          1
4       RB-MON          1
5       RB-SLB          4
6       RB-SRB        414
7          SAB          5
8      SAB-SRB          3
9          SLB        187
10     SLB-MON          6
11      SLB-RB         17
12     SLB-SRB        171
13         SRB       1129
14     SRB-MON          3
15      SRB-RB        260
16     SRB-SAB          2
17     SRB-SLB         21
"""

filtered_df = df5[df5['BreedDelPro'] == "02 SLB"]
frequency_table = filtered_df.groupby(['SireBreedKok']).size().reset_index(name='Frequency')
print(frequency_table)
"""
SireBreedKok  Frequency
0           ANG        144
1           BGA          1
2            BL          4
3           CHA         45
4           HER        135
5           KRS          1
6           LIM        188
7           MON         15
8            RB         27
9        RB-SRB          1
10          SIM          6
11          SJB          4
12          SLB       6286
13          SRB         73
14          WAG          5
"""
frequency_table = filtered_df.groupby(['DamBreedKok']).size().reset_index(name='Frequency')
print(frequency_table)
"""
DamBreedKok  Frequency
0      HER-SLB          2
1          KRS          1
2      KRS-SLB          2
3      MON-SLB         17
4      MON-SRB          3
5           RB          4
6       RB-MON          3
7       RB-SLB          7
8       RB-SRB         13
9      SAB-SRB          1
10     SIM-SRB          1
11         SJB          2
12     SJB-SLB          1
13         SLB       6501
14     SLB-BSW          1
15     SLB-HER          5
16     SLB-KRS         13
17     SLB-MON         40
18      SLB-RB         24
19     SLB-SAB          5
20     SLB-SJB          1
21     SLB-SKB          1
22     SLB-SRB        290
23         SRB         50
24     SRB-KRS          2
25     SRB-MON          9
26      SRB-RB          9
27     SRB-SLB         92
"""